<a href="https://colab.research.google.com/github/rhythmgoyal11/Momentum/blob/main/Momentum_Str_RhythmGoyal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Momentum Strategy - Rhythm Goyal

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tickers = pd.read_csv('/content/drive/MyDrive/Momentum Strategy/ind_niftytotalmarket_list_rebalance_sep25.csv')
tickers['NSSymbol'] = tickers['Symbol'] + ".NS"

tickers_list = tickers['NSSymbol'].tolist()

tickers

,Company Name,Industry,Symbol,Series,ISIN Code,NSSymbol
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038,360ONE.NS
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017,3MINDIA.NS
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022,ABB.NS
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025,ACC.NS
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025,ACMESOLAR.NS
...,...,...,...,...,...,...
746,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027,ZENSARTECH.NS
747,Zydus Lifesciences Ltd.,Healthcare,ZYDUSLIFE,EQ,INE010B01027,ZYDUSLIFE.NS
748,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01028,ZYDUSWELL.NS
749,eClerx Services Ltd.,Services,ECLERX,EQ,INE738I01010,ECLERX.NS


In [ ]:
today = datetime.today()

start_12m = today - timedelta(days=365)
start_6m = today - timedelta(days=182)
end = today - timedelta(days=30)

ema_lookback = today - timedelta(days=730)

start_date = ema_lookback
end_date = today

def get_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
    return df

all_stock_data = {}

for ticker in tickers_list:
    try:
        df = get_data(ticker, start_date, end_date)
        if not df.empty:
            all_stock_data[ticker] = df[['Close', 'Volume']]
        else:
            print(f"No data for {ticker}")
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

filtered_stocks = {}
for ticker, df in all_stock_data.items():
    if len(df) < 200: # Need at least 200 data points for EMA
        continue

    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    if df['Close'].iloc[-1].item() > df['EMA_200'].iloc[-1].item():
        filtered_stocks[ticker] = df.loc[(df.index >= start_12m) & (df.index <= end)].copy()




ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DUMMYDBRLT.NS"}}}
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['DUMMYDBRLT.NS']: YFTzMissingError('possibly delisted; no timezone found')


No data for DUMMYDBRLT.NS


In [ ]:
momentum_scores = []

# check to see if filtered_stocks is empty
if not filtered_stocks:
    print("No stocks met the filtering criteria (length > 200 and Close > EMA_200). Cannot calculate momentum.")
else:
    for ticker, df in filtered_stocks.items():
        try:
            if len(df) < len(df.loc[(df.index >= start_12m) & (df.index <= end)]):
                 print(f"Not enough data points for {ticker} in the lookback window.")
                 continue

            df = df.copy()

            df = df.dropna()


            df['Daily Return'] = df['Close'].pct_change()


            df['Gross Return'] = 1 + df['Daily Return']

            # 12-month window: T−365 to T−30 days
            mask_12m = (df.index >= start_12m) & (df.index <= end)
            gross_returns_12m = df.loc[mask_12m, 'Gross Return']

            if not gross_returns_12m.empty:
                cumulative_return_12m = gross_returns_12m.prod() - 1  # total return
            else:
                cumulative_return_12m = 0
                print(f"No data for {ticker} in 12-month window.")

            mask_6m = (df.index >= start_6m) & (df.index <= end)
            gross_returns_6m = df.loc[mask_6m, 'Gross Return']

            if not gross_returns_6m.empty:
                 cumulative_return_6m = gross_returns_6m.prod() - 1
            else:
                cumulative_return_6m = 0
                print(f"No data for {ticker} in 6-month window.")

            composite_momentum = 0.5*cumulative_return_12m + 0.5*cumulative_return_6m


            if not pd.isna(composite_momentum):
                 momentum_scores.append({
                    'Ticker': ticker,
                    '6M Momentum': cumulative_return_6m,
                    '12M Momentum': cumulative_return_12m,
                    'Composite Momentum': composite_momentum
                })
            else:
                 print(f"Skipping {ticker} due to NaN composite momentum.")


        except Exception as e:
            print(f"Error calculating momentum for {ticker}: {e}")


if momentum_scores:
    momentum_df = pd.DataFrame(momentum_scores)

    # Rank based on Composite Momentum (higher is better)
    momentum_df = momentum_df.sort_values(by='Composite Momentum', ascending=False).reset_index(drop=True)

    # View top 40
    display(momentum_df.head(40))
else:
    print("No momentum scores calculated.")
    momentum_df = pd.DataFrame()


,Ticker,6M Momentum,12M Momentum,Composite Momentum
0,LUMAXTECH.NS,1.579428,1.503605,1.541516
1,GABRIEL.NS,1.306852,1.707476,1.507164
2,BANCOINDIA.NS,1.345792,1.410783,1.378288
3,NETWEB.NS,1.868140,0.497128,1.182634
4,BLACKBUCK.NS,0.503678,1.436647,0.970163
5,IXIGO.NS,0.950733,0.896575,0.923654
6,FORCEMOT.NS,0.697182,1.134022,0.915602
7,GMDCLTD.NS,1.075022,0.694838,0.884930
8,SUNDARMHLD.NS,0.893153,0.820350,0.856751
9,GVT&D.NS,0.988009,0.707220,0.847615


In [ ]:
top_40_momentum = momentum_df.head(40).copy()

fiftytwo_week_high_data = []

for index, row in top_40_momentum.iterrows():
    ticker = row['Ticker']

    # need data for the last 52 weeks from today
    end_date_52w = today # Use today as the end date for 52-week high calculation
    start_date_52w = today - timedelta(days=365) # Approximate 52 weeks

    stock_data = all_stock_data.get(ticker)

    if stock_data is None or stock_data.empty:
        print(f"No data found for {ticker} to calculate 52-week high.")
        continue

    # Filter data for the 52-week window
    fiftytwo_week_data = stock_data.loc[(stock_data.index >= start_date_52w) & (stock_data.index <= end_date_52w)]

    if fiftytwo_week_data.empty:
        print(f"No data in the 52-week window for {ticker}.")
        continue

    # Calculate the 52-week high
    fiftytwo_week_high = fiftytwo_week_data['Close'].max().item()

    # Get the current closing price (most recent price available)
    current_price = stock_data['Close'].iloc[-1].item()

    # Calculate percentage difference from 52-week high
    # Avoid division by zero if 52-week high is 0 (unlikely for stocks but good practice)
    if fiftytwo_week_high > 0:
        pct_from_fiftytwo_week_high = ((fiftytwo_week_high - current_price) / fiftytwo_week_high) * 100
    else:
        pct_from_fiftytwo_week_high = 100 # If high is 0, assume 100% away (or some other large value)

    fiftytwo_week_high_data.append({
        'Ticker': ticker,
        '52 Week High': fiftytwo_week_high,
        'Current Price': current_price,
        'Pct From 52 Week High': pct_from_fiftytwo_week_high
    })

fiftytwo_week_high_df = pd.DataFrame(fiftytwo_week_high_data)



In [ ]:
from numpy import sign

# Get top 40 stocks by composite momentum
top_40 = momentum_df.head(40).copy() # Create a copy to avoid SettingWithCopyWarning later

# Initialize list to hold quality scores
quality_scores = []

# Loop through each of the top 40 to compute smoothness
for i, row in top_40.iterrows():
    ticker = row['Ticker']
    df = all_stock_data.get(ticker)

    if df is None or df.empty:
        continue

    df = df.copy()
    lookback_df = df.loc[(df.index >= start_12m) & (df.index <= end)]
    lookback_df['Daily Return'] = lookback_df['Close'].pct_change()
    lookback_df = lookback_df.dropna()

    total_days = len(lookback_df)
    if total_days == 0:
        continue

    num_pos = (lookback_df['Daily Return'] > 0).sum()
    num_neg = (lookback_df['Daily Return'] < 0).sum()

    pct_pos = num_pos / total_days
    pct_neg = num_neg / total_days

    # Smoothness formula
    past_return = row['Composite Momentum']

    # Handle the case where past_return is 0 or very close to 0
    # The sign function will return 0 for 0, which might be intended,
    # but if you want to avoid dividing by zero or having a smoothness of 0
    # when the return is flat, you might adjust this.
    # For now, we'll keep the original sign logic.

    smoothness = sign(past_return) * (pct_neg - pct_pos)

    quality_scores.append({
        'Ticker': ticker,
        'Composite Momentum': past_return,
        'Smoothness Score': smoothness
    })

# Convert results to DataFrame
quality_df = pd.DataFrame(quality_scores)

# --- Add the filter for negative smoothness scores ---
quality_df = quality_df[quality_df['Smoothness Score'] < 0].copy()



# Merge this data with the quality_df
# This will add the 52-week high related columns to the quality_df
quality_df = pd.merge(quality_df, fiftytwo_week_high_df[['Ticker', 'Pct From 52 Week High']], on='Ticker', how='left')

# Handle cases where a stock in quality_df didn't have 52-week high data
# (though this should be rare if they were in top 40 momentum and had data)
# You might want to drop these rows or fill NaN
quality_df.dropna(subset=['Pct From 52 Week High'], inplace=True)

# Now, proceed with ranking based on Pct From 52 Week High
# Lower percentage is better (closer to the high)
quality_df['52 Week High Rank'] = quality_df['Pct From 52 Week High'].rank(ascending=True) # Use ascending=True


# Ensure both scores are numeric, handle potential NaNs if necessary
# (The current logic should prevent NaNs from smoothness, but it's good practice)
if quality_df.empty:
  print("No stocks with negative smoothness score.")

else:
  quality_df.dropna(subset=['Composite Momentum', 'Smoothness Score'], inplace=True)

# Rank Composite Momentum (higher is better)
  quality_df['Momentum Rank'] = quality_df['Composite Momentum'].rank(ascending=False)

# Rank Smoothness Score (lower is better, as pct_neg - pct_pos closer to 0 is smoother)
  quality_df['Smoothness Rank'] = quality_df['Smoothness Score'].rank(ascending=True)

# Calculate combined rank with 0.5 weightage to each
  quality_df['Combined Rank'] = (0.6 * quality_df['Momentum Rank'] +
                             0.25 * quality_df['Smoothness Rank'] +
                             0.15 * quality_df['52 Week High Rank'])

# Sort by the combined rank (lower combined rank is better)
  top_40_final = quality_df.sort_values(by='Combined Rank', ascending=True).reset_index(drop=True)

# Display the final ranked list of all 40 stocks
  display(top_40_final)

# --- End of modifications ---

/tmp/ipython-input-3775765839.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookback_df['Daily Return'] = lookback_df['Close'].pct_change()
/tmp/ipython-input-3775765839.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookback_df['Daily Return'] = lookback_df['Close'].pct_change()
/tmp/ipython-input-3775765839.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,Ticker,Composite Momentum,Smoothness Score,Pct From 52 Week High,52 Week High Rank,Momentum Rank,Smoothness Rank,Combined Rank
0,BLACKBUCK.NS,0.970163,-0.088785,6.843901,16.0,2.0,6.0,5.100
1,LUMAXTECH.NS,1.541516,-0.088106,14.160424,25.0,1.0,7.0,6.100
2,CARTRADE.NS,0.822944,-0.123348,3.674405,10.0,7.0,3.0,6.450
3,LTF.NS,0.672090,-0.158590,1.347808,5.0,11.0,1.0,7.600
4,IMFA.NS,0.836937,-0.066079,6.767521,15.0,6.0,9.0,8.100
5,AIIL.NS,0.779495,-0.106195,11.529017,23.0,8.0,4.0,9.250
6,GVT&D.NS,0.847615,-0.017621,3.359515,9.0,5.0,22.0,9.850
7,IXIGO.NS,0.923654,-0.039648,18.894351,26.0,3.0,17.0,9.950
8,FORCEMOT.NS,0.915602,-0.044053,21.743856,27.0,4.0,15.0,10.200
9,LAURUSLABS.NS,0.579237,-0.154185,0.000000,2.0,16.0,2.0,10.400


In [ ]:
from google.colab import auth
from google.auth import default
import gspread
import pandas as pd
from datetime import date

# Authenticate to Google Drive and Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Specify the name of your Google Sheet
mom_str_sheet = 'Momentum Strategy Results'  # Replace with your desired sheet name

try:
    # Try to open the spreadsheet
    sh = gc.open(mom_str_sheet)
    print(f"Successfully opened sheet: {mom_str_sheet}")

    # Select the first worksheet
    worksheet = sh.sheet1
    print("Selected sheet1")

    # Clear existing data
    worksheet.clear()
    print("Cleared existing data")

except gspread.SpreadsheetNotFound:
    # If the spreadsheet doesn't exist, create it
    print(f"Sheet '{mom_str_sheet}' not found. Creating new sheet.")
    sh = gc.create(mom_str_sheet)
    print(f"Successfully created sheet: {mom_str_sheet}")

    # Get the first worksheet
    worksheet = sh.sheet1
    print("Selected sheet1")

except Exception as e:
    print(f"An error occurred while accessing the sheet: {e}")
    # Handle other potential errors during sheet access


# Assuming 'top_40_final' is the DataFrame you want to export
# Add a check to ensure top_40_final exists and is not empty
if 'top_40_final' in locals() and not top_40_final.empty:
    try:
        # Get today's date
        today_date = date.today().strftime('%Y-%m-%d') # Format the date as YYYY-MM-DD

        # Convert the DataFrame to a list of lists (including headers)
        # Add the 'Date' header at the beginning
        headers = ['Date'] + top_40_final.columns.values.tolist()

        # Add today's date as the first element to each row of data
        data_rows = [[today_date] + row for row in top_40_final.values.tolist()]

        # Combine headers and data rows
        data_to_export = [headers] + data_rows

        # Export the data to the Google Sheet
        worksheet.update(data_to_export)
        print("Successfully exported data to Google Sheet with Date column.")

    except Exception as e:
        print(f"An error occurred while exporting data: {e}")
else:
    print("The 'top_40_final' DataFrame is not available or is empty. No data exported.")

Successfully opened sheet: Momentum Strategy Results
Selected sheet1
Cleared existing data
Successfully exported data to Google Sheet with Date column.
